<a href="https://colab.research.google.com/github/fazal735/DL_A2/blob/main/DL_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import torchvision
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train_data_dir = '/content/drive/MyDrive/inaturalist_12K/train'
test_data_dir = '/content/drive/MyDrive/inaturalist_12K/val'

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:

# train data loading
def train_data(train_data_dir,data_augmentation):
  size=transforms.Resize((224,224))
  to_tensor=transforms.ToTensor()
  #check again-autogenerated
  normalize=transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
  crop=transforms.RandomResizedCrop(224)
  flip=transforms.RandomHorizontalFlip()
  #try changing the values
  color=transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
  rotation=transforms.RandomRotation(20)


  #augmentation of image
  if data_augmentation == 'Yes':
        transform_img = transforms.Compose([crop,flip,color, rotation, to_tensor,normalize]) # Data transformations

  else:
      transform_img = transforms.Compose([size,to_tensor, normalize ])



  #data fetchiing
  training_data=torchvision.datasets.ImageFolder(train_data_dir, transform=transform_img)


  train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
  train_sampler = SubsetRandomSampler(train_index)
  val_sampler = SubsetRandomSampler(val_index)

  train_data=DataLoader(training_data,batch_size=32,sampler=train_sampler)
  validation_data = DataLoader(training_data, batch_size=32, sampler=val_sampler)
  print('Train data size:', len(train_data))
  print('Validation data size:', len(validation_data))

  return train_data,validation_data

#test data loading
def test_data(test_data_dir,data_augmentation):
  size=transforms.Resize((224,224))
  to_tensor=transforms.ToTensor()
  #check again-autogenerated
  normalize=transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])


  #augmentation of image
  data_transform=transforms.Compose([size,to_tensor,normalize])

  #data fetching
  test_data=torchvision.datasets.ImageFolder(test_data_dir,transform=data_transform)
  test_data=DataLoader(test_data,batch_size=32)


  return test_data

model definition

In [9]:

class model(nn.Module):
  def __init__(self,num_filters=[32,64,128,256,512],filter_size=[3,3,5,5,7],activation=nn.ReLU(),
               stride=1, padding=1, pool_size=(2,2),fc_size=512,nom_o_classes=10,
               dropout=0,in_channels=3,batch_norm='YES'):
    super(model,self).__init__()
    self.num_filters=num_filters
    self.filter_size=filter_size
    self.activation=activation
    self.stride=stride
    self.padding=padding
    self.pool_size=pool_size
    self.fc_size=fc_size
    self.nom_o_classes=nom_o_classes
    self.dropout=dropout
    self.channels=in_channels


    def image_size(img_w,filter_size,padding,stride):
      return ((img_w-filter_size+2*padding)/stride+1)*0.5


    #layers of convolution
    #layer1
    self.conv_layer1=nn.Conv2d(self.channels,self.num_filters[0], stride=self.stride, padding=self.padding,
                               kernel_size=self.filter_size[0])
    self.batch_norm1=nn.BatchNorm2d(self.num_filters[0])
    #self.dropout1=nn.Dropout2d(self.dropout)

    img_size1=image_size(224,self.filter_size[0],self.padding,self.stride)

    #layer2
    self.conv_layer2=nn.Conv2d(self.num_filters[0],self.num_filters[1], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[1])
    self.batch_norm2=nn.BatchNorm2d(self.num_filters[1])
    self.dropout2=nn.Dropout2d(self.dropout)

    img_size2=image_size(img_size1,self.filter_size[1],self.padding,self.stride)

    #layer3
    self.conv_layer3=nn.Conv2d(self.num_filters[1],self.num_filters[2], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[2])
    self.batch_norm3=nn.BatchNorm2d(self.num_filters[2])
    self.dropout3=nn.Dropout2d(self.dropout)

    img_size3=image_size(img_size2,self.filter_size[2],self.padding,self.stride)

    #layer4
    self.conv_layer4=nn.Conv2d(self.num_filters[2],self.num_filters[3], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[3])
    self.batch_norm4=nn.BatchNorm2d(self.num_filters[3])
    self.dropout4=nn.Dropout2d(self.dropout)

    img_size4=image_size(img_size3,self.filter_size[3],self.padding,self.stride)

    #layer5
    self.conv_layer5=nn.Conv2d(self.num_filters[3],self.num_filters[4], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[4])
    self.batch_norm5=nn.BatchNorm2d(self.num_filters[4])
    self.dropout5=nn.Dropout2d(self.dropout)

    img_size5=int(image_size(img_size4,self.filter_size[4],self.padding,self.stride))


    self.pool=nn.MaxPool2d(self.pool_size,stride=2)

    self.dropout_layer = nn.Dropout1d(self.dropout)

    # Define fully connected layer
    self.fc = nn.Linear(self.num_filters[4] * (img_size5 ** 2), self.fc_size)
    self.fc_bn = nn.BatchNorm1d(self.fc_size)  # Batch normalization for fully connected layer

    # Output layer
    self.output_layer = nn.Linear(self.fc_size, self.nom_o_classes)

    # forward
  def forward(self,x):
    #layer1
    x=self.conv_layer1(x)
    x=self.activation(x)
    x=self.pool(x)
    #x=self.dropout1(x)
    x=self.batch_norm1(x)

      #layer2
    x=self.conv_layer2(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout2(x)
    x=self.batch_norm2(x)

    #layer3
    x=self.conv_layer3(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout3(x)
    x=self.batch_norm3(x)

    #layer4
    x=self.conv_layer4(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout4(x)
    x=self.batch_norm4(x)

    #layer5
    x=self.conv_layer5(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout5(x)
    x=self.batch_norm5(x)

    x = torch.flatten(x, 1)
    x = self.fc(x)
    x = self.fc_bn(x)
    x = self.activation(x)
    x = self.dropout_layer(x)
    x = self.output_layer(x)

    return x

model1=model()
model1.to(device)
print(model1)

model(
  (activation): ReLU()
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout2d(p=0, inplace=False)
  (conv_layer3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout2d(p=0, inplace=False)
  (conv_layer4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (batch_norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout2d(p=0, inplace=False)
  (conv_layer5): Conv2d(256, 512, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
  (batch_norm5): BatchNorm2d(

In [10]:
epochs=100
learning_rate=0.1

loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model1.parameters(),lr=learning_rate)

model training function

In [11]:
# loss_metric=nn.CrossEntropyLoss()
# optimizer=torch.optim.SGD(model1.parameters(),lr=learning_rate)

# def training(model1,data):

#   model1.train(True)
#   training_loss=0.0
#   true_label=0
#   total_train=0

#   for input, label in data:
#     input = input.to(device)
#     label = label.to(device)
#     optimizer.zero_grad()

#     output=model1(input)
#     loss=loss_metric(output,label)
#     loss.backward()
#     optimizer.step()
#     training_loss += loss.item()
#     _,predicted=torch.max(output.data,1)
#     total_train += label.size(0)
#     true_label += (predicted==label).sum().item()

#   train_accuracy=100*true_label/total_train
#   return train_accuracy,training_loss,model1


model testing function(on validation data)

In [12]:
loss_metric=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model1.parameters(),lr=learning_rate)

def training(model1, data, epoch=0):
    model1.train(True)
    training_loss = 0.0
    true_label = 0
    total_train = 0

    # Add batch tracking
    total_batches = len(data)

    for batch_idx, (input, label) in enumerate(data):
        input = input.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        output = model1(input)
        loss = loss_metric(output, label)
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_train += label.size(0)
        true_label += (predicted == label).sum().item()

        # Print progress every few batches
        if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == total_batches:
            print(f'Epoch: {epoch} [{batch_idx+1}/{total_batches}] Loss: {loss.item():.4f}')

    train_accuracy = 100 * true_label / total_train
    avg_loss = training_loss / total_batches

    print(f"Training completed - Accuracy: {train_accuracy:.2f}%, Avg Loss: {avg_loss:.4f}")

    return train_accuracy, training_loss, model1

In [13]:
def test_on_valid_data(model, test_data):
    model.eval()

    correct_label = 0
    total_label = 0
    with torch.no_grad():
        for img, label in test_data:
            img, label = img.to(device), label.to(device)
            output = model(img)

            _, pred = torch.max(output, 1)
            correct_label += (pred == label).sum().item()
            total_label += label.size(0)

    valid_accuracy = 100 * correct_label / total_label
    print(f'Validation Accuracy: {valid_accuracy:.2f}%')
    return valid_accuracy


In [14]:
def model_train_val(model, train_data, val_data,epochs,device=device):

    # criterion = nn.CrossEntropyLoss()

    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  print("model train val")
  for epoch in range(epochs):
    train_accuracy,avg_loss,model1 = training(model, train_data)
    # Print training loss and accuracy
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    val_accuracy = test_on_valid_data(model, val_data)
    # Print validation accuracy
    print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')


    wandb.log({'Train loss': avg_loss})
    wandb.log({'Train accuracy': train_accuracy})

    wandb.log({'val_accuracy': val_accuracy})
    wandb.log({'epoch': epoch})

def model_train_test(model, train_data, test_data,epochs):

  for epoch in range(epochs):
    train_accuracy,avg_loss,model1 = training(model, train_data)
    # Print training loss and accuracy
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    val_accuracy = test_on_valid_data(model, val_data)
    # Print validation accuracy
    print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')


    wandb.log({'Train loss': avg_loss})
    wandb.log({'Train accuracy': train_accuracy})

    wandb.log({'val_accuracy': val_accuracy})
    wandb.log({'epoch': epoch})




In [15]:

import time
import traceback

def model_train_val(model, train_data, val_data, epochs, device=device):
    print("Starting model_train_val function")

    # Define criterion and optimizer here
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Pass these to your training function if needed
    # Or ensure global variables are properly synchronized

    print(f"Beginning training for {epochs} epochs")
    for epoch in range(epochs):
        print(f"Starting epoch {epoch+1}/{epochs}")

        # Capture start time to monitor duration
        start_time = time.time()

        try:
            print("Calling training function...")
            train_accuracy, avg_loss, model = training(model, train_data)
            print(f"Training complete for epoch {epoch+1}")

            # Print training metrics
            print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

            print("Starting validation...")
            val_accuracy = test_on_valid_data(model, val_data)
            print(f"Validation complete for epoch {epoch+1}")

            # Print validation metrics
            print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')

            # Calculate epoch duration
            epoch_time = time.time() - start_time
            print(f"Epoch {epoch+1} completed in {epoch_time:.2f} seconds")

            # Log to wandb with try-except to catch any logging errors
            try:
                print("Logging to wandb...")
                wandb.log({
                    'epoch': epoch,
                    'train_loss': avg_loss,
                    'train_accuracy': train_accuracy,
                    'val_accuracy': val_accuracy,
                    'epoch_time': epoch_time
                })
                print("Successfully logged to wandb")
            except Exception as e:
                print(f"Error logging to wandb: {str(e)}")

        except Exception as e:
            print(f"Error during epoch {epoch+1}: {str(e)}")
            traceback.print_exc()  # This will print the full stack trace

    print("Model training and validation completed")
    return model




In [16]:
!pip install wandb

e4d0a8c3ccaf2534e9ab91c659e420ba5114533f

> Add blockquote



In [17]:

import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mfazal735 (mfazal735-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:


sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3],[3,5,5,7,7],[3,5,3,5,7],[5,5,5,5,5]]#,[7,7,7,7,7]]
        },
        'dropout': {
            'values': [0.3, 0.2]
        },
        'activation': {
            'values': [ 'relu','mish','silu', 'gelu',]
        },
        'num_dense':{
            'values': [128, 256]
        },
        'batch_norm':{
            'values': ['Yes','No']
        },
        'filter_org':{
            'values': [[128,128,64,64,32],[32,64,128,256,512],[32,32,32,32,32],[32,64,64,128,128]]
        },
        'learning_rate':{
            'values': [0.001,0.0001]
        },
        'optimizer':{
            'values': ['Adam','SGD']
        },
        'data_aug': {
            'values': ['No', 'Yes']
        }

    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL A2')

Create sweep with ID: 2hsjgo25
Sweep URL: https://wandb.ai/mfazal735-iit-madras-foundation/DL%20A2/sweeps/2hsjgo25


In [ ]:
def main():

    with wandb.init(project='DL A2') as run:
        run_name="ks"+str(wandb.config.kernel_size)+"ac-"+(wandb.config.activation)+"_drop-"+str(wandb.config.dropout)+"_fs-"+str(wandb.config.filter_org)+"_bn-"+str(wandb.config.batch_norm)+"_dence-"+str(wandb.config.num_dense)
        wandb.run.name=run_name


        if  wandb.config.activation == 'relu':
            activ=nn.ReLU()
        elif wandb.config.activation == 'gelu':
            activ=nn.GELU()
        elif wandb.config.activation == 'silu':
            activ=nn.SiLU()
        elif wandb.config.activation == 'mish':
            activ=nn.Mish()

        model_= model(num_filters=wandb.config.filter_org, filter_size=wandb.config.kernel_size,
                      activation=activ, stride=1,padding=1, pool_size=(2,2), fc_size=wandb.config.num_dense,
                      nom_o_classes=10,dropout = wandb.config.dropout).to(device)

        train, validation = train_data(train_data_dir,data_augmentation= wandb.config.data_aug)

        model_train_val(model=model_, train_data=train, val_data=validation, epochs = 10)

wandb.agent(sweep_id, function= main,count= 5)


wandb: Agent Starting Run: tez1xjb0 with config:
wandb: 	activation: mish
wandb: 	batch_norm: No
wandb: 	data_aug: Yes
wandb: 	dropout: 0.3
wandb: 	filter_org: [32, 64, 64, 128, 128]
wandb: 	kernel_size: [3, 5, 3, 5, 7]
wandb: 	learning_rate: 0.0001
wandb: 	num_dense: 128
wandb: 	optimizer: Adam


Train data size: 250
Validation data size: 63
Starting model_train_val function
Beginning training for 10 epochs
Starting epoch 1/10
Calling training function...
Epoch: 0 [10/250] Loss: 2.4601
Epoch: 0 [20/250] Loss: 2.4936


In [ ]:


sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3],[3,5,5,7,7],[3,5,3,5,7],[5,5,5,5,5]]#,[7,7,7,7,7]]
        },
        'dropout': {
            'values': [0.3, 0.2]
        },
        'activation': {
            'values': [ 'relu','mish','silu', 'gelu',]
        },
        'num_dense':{
            'values': [128, 256]
        },
        'batch_norm':{
            'values': ['Yes','No']
        },
        'filter_org':{
            'values': [[128,128,64,64,32],[32,64,128,256,512],[32,32,32,32,32],[32,64,64,128,128]]
        },
        'learning_rate':{
            'values': [0.001,0.0001]
        },
        'optimizer':{
            'values': ['Adam','SGD']
        },
        'data_aug': {
            'values': ['No', 'Yes']
        }

    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL A2')

In [ ]:
def main():

    with wandb.init(project='DL A2') as run:
        run_name="ks"+str(wandb.config.kernel_size)+"ac-"+(wandb.config.activation)+"_drop-"+str(wandb.config.dropout)+"_fs-"+str(wandb.config.filter_org)+"_bn-"+str(wandb.config.batch_norm)+"_dence-"+str(wandb.config.num_dense)
        wandb.run.name=run_name


        if  wandb.config.activation == 'relu':
            activ=nn.ReLU()
        elif wandb.config.activation == 'gelu':
            activ=nn.GELU()
        elif wandb.config.activation == 'silu':
            activ=nn.SiLU()
        elif wandb.config.activation == 'mish':
            activ=nn.Mish()

        model_= model(num_filters=wandb.config.filter_org, filter_size=wandb.config.kernel_size,
                      activation=activ, stride=1,padding=1, pool_size=(2,2), fc_size=wandb.config.num_dense,
                      nom_o_classes=10,dropout = wandb.config.dropout).to(device)

        train, validation = train_data(train_data_dir,data_augmentation= wandb.config.data_aug)

        model_train_val(model=model_, train_data=train, val_data=validation, epochs = 10)

        model_train_test(model=model_, train_data=test, val_data=validation, epochs = 10)

wandb.agent(sweep_id, function= main,count= 5)

In [3]:
import imageio.v2
import os
import cv2
from PIL import Image
import random

def img_plot(model):
    #For plotting random Images
    classes_list = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
    # Directory of the test data
    dir = '/content/drive/MyDrive/inaturalist_12K/val'
    images = []  # List to store the images
    labels = []  #  List to store the labels

    for label, name in enumerate(classes_list):
      images_list = os.listdir(dir+'/'+name)
      img_names = []
      for j in range(3):
        index = random.randint(0, 199)
        img_names.append(images_list[index])
      for image_name in img_names:
          image = imageio.v2.imread(dir+'/'+name+'/'+image_name)
          if np.ndim(image) == 3:
            images.append(cv2.resize(image, (224,224)))
            labels.append(classes_list[label])

    arr = np.array(images)
    arr = (arr/255.0).astype('float32')
    transp_arr = np.transpose(arr, (0, 3, 1, 2))
    tensor1 = torch.tensor(transp_arr).to(device)
    y_pred = model(tensor1)
    y_pred_label = torch.argmax(y_pred, 1)
    dictn = {0:'Amphibia',1:'Animalia',2:'Arachnida',3:'Aves',4:'Fungi',
               5:'Insecta',6:'Mammalia',7:'Mollusca',8:'Plantae',9:'Reptilia'}

    fig, axes = plt.subplots(10, 3, figsize=(12, 20))
    for i in range(10):
        for j in range(3):
            ind = i * 3 + j
            if ind < 30:
                axes[i, j].imshow(images[ind])
                axes[i, j].set_title(f'True: {labels[ind]}, Predicted: {dictn[y_pred_label[idx].item()]}')
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.show()
    fig.savefig('pictures.png', bbox_inches='tight')
    # Convert the saved image to Image
    image = Image.open('pictures.png')
    fig.canvas.draw()
    image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())

img_plot(model_)
    # wandb.log({"images": [wandb.Image(image, caption="Test image prediction")]})

NameError: name 'model_' is not defined